# Assignment 2: Flexibility and Service Level of a JIT Production System
Yang Zhang 1726414

Task 1 : 

1.import all needed packages

In [1]:
import numpy as np
import pandas as pd
import docplex
from docplex.mp.model import Model

2.Create Model Instance

In [2]:
model = Model(name="JIT System")

3.Define all sets

In [3]:
time = [1,2,3,4,5,6,7,8,9,10] # planning periods
T = 10 #total planning horizon
d_t = [310,380,400,120,200,200,400,1400,70,60]
c_NC = 10
c_OC = 15
c_h = 4
c_b = 6
NC_max = 354
OC_max = 120
SL = 0.95
GTL = 1 #lower bound of the goal time window
GTU = 1 #Upper bound of the goal time window

4.Define the decision variables

In [4]:
X = model.integer_var_dict(time, name = "Prod") # production quantity that delivered on the shipping day
I = model.integer_var_dict(time, name = "Inv") # inventory level -- the amount that production exceeds demand
O = model.integer_var_dict(time, name = "Over") # additional personnel capacity
D = model.integer_var_dict(((m,n) for m in time for n in time), name= "Deliver") #m represents the expected shipping day, n represents the day it is shipped out
L = model.integer_var_dict(time, name= "Backperday") # L records the quantity of backorders per day----orders past the goal time window per day
B = model.integer_var_dict(time, name= "Backaccmu") #B records the accumulated quantity of backorders
NC = model.integer_var(name = "normalcap")  #normal capacity 

5.Define all constraints

In [5]:
#Inventory Balance
model.add_constraints(I[t-1] + X[t] + O[t] - model.sum(D[m,t] for m in time) == I[t] for t in range(2,T+1)) 
model.add_constraint(X[1] + O[1] - model.sum(D[m,1] for m in time) == I[1])

#Demand fulfilled constraint
model.add_constraints(model.sum(D[m,n]for n in time) == d_t[m-1]for m in time)

#Service Level constraint
model.add_constraints(L[m] == model.sum(D[m,n] for n in range(m + GTU + 1,T+1))for m in range(1,T - GTU)) # the number of items which is shipped out of the upper limit of the goal time window
model.add_constraints(L[t] <= (1-SL)* d_t[t-1] for t in time)

#Goal time window constraint
model.add_constraints(D[m,n] == 0 for m in range(GTL+2,T+1) for n in range(1,m-GTL))#The actual shipping date cannot be earlier that goal time window

#Backorder constraint 
model.add_constraints(B[t] == model.sum(d_t[m-1] for m in range(1,t-GTU))-model.sum(D[m,n] for m in range(1,t-GTU) for n in range(1,t)) for t in range(GTU+2,T+1))

#Capacity Constraints  
model.add_constraint(NC<= NC_max) 
model.add_constraints(X[t] <= NC for t in time)
model.add_constraints(O[t] <= OC_max for t in time)

#Non-negative constriant
model.add_constraints(X[t] >= 0 for t in time)
model.add_constraints(I[t] >= 0 for t in time)
model.add_constraints(O[t] >= 0 for t in time)  
model.add_constraints(B[t] >= 0 for t in time)  

[docplex.mp.LinearConstraint[](Backaccmu_1,GE,0),
 docplex.mp.LinearConstraint[](Backaccmu_2,GE,0),
 docplex.mp.LinearConstraint[](Backaccmu_3,GE,0),
 docplex.mp.LinearConstraint[](Backaccmu_4,GE,0),
 docplex.mp.LinearConstraint[](Backaccmu_5,GE,0),
 docplex.mp.LinearConstraint[](Backaccmu_6,GE,0),
 docplex.mp.LinearConstraint[](Backaccmu_7,GE,0),
 docplex.mp.LinearConstraint[](Backaccmu_8,GE,0),
 docplex.mp.LinearConstraint[](Backaccmu_9,GE,0),
 docplex.mp.LinearConstraint[](Backaccmu_10,GE,0)]

6.Define linear expressions

In [6]:
Z = model.sum(c_h * I[t] + NC * c_NC + O[t] * c_OC + B[t] * c_b for t in time)

7.Define the objective function

In [7]:
model.minimize(Z)

8.Solve the model

In [8]:
model.solve()
model.solve_details

docplex.mp.SolveDetails(time=0.031,status='integer optimal solution')

9.Print the results

In [9]:
header_1 = ['Time','Optimal normal capacities', 'Assigned over capacities']
df_1 = pd.DataFrame(columns = header_1)
for t in time:
    df_1.loc[len(df_1), 'Time'] = t
    df_1.loc[len(df_1)-1, 'Optimal normal capacities'] = NC.solution_value
    df_1.loc[len(df_1)-1, 'Assigned over capacities'] = O[t].solution_value
print(df_1)
print("\n")    
print("The objective value is ", model.objective_value) # costs are expressed in hundreds of euro!

  Time Optimal normal capacities Assigned over capacities
0    1                       332                        0
1    2                       332                        0
2    3                       332                        0
3    4                       332                        0
4    5                       332                        0
5    6                       332                        0
6    7                       332                       94
7    8                       332                      120
8    9                       332                      120
9   10                       332                        0


The objective value is  39046.0


In [10]:
header_2 = ['Time','Daily inventory', 'Backorders', 'Utilization']
df_2 = pd.DataFrame(columns = header_2)
for t in time:
    df_2.loc[len(df_2), 'Time'] = t
    df_2.loc[len(df_2)-1,'Daily inventory'] = I[t].solution_value
    df_2.loc[len(df_2)-1, 'Backorders'] = B[t].solution_value
    df_2.loc[len(df_2)-1, 'Utilization'] = X[t].solution_value/NC.solution_value
print(df_2)
print("\n")    

for t in time:
    df_2.loc[len(df_2), 'Time'] = t
    df_2.loc[len(df_2)-1,'Daily inventory'] = I[t].solution_value
    df_2.loc[len(df_2)-1, 'Backorders'] = L[t].solution_value

  Time Daily inventory Backorders Utilization
0    1               0          0           1
1    2               0          0           1
2    3               0          0           1
3    4               0          0           1
4    5              50          0           1
5    6               0          0           1
6    7               0          0           1
7    8               0          0           1
8    9               0         18           1
9   10               0         88    0.656627




In [11]:
#header_3 = ['Actual shipping date']
#Index_3 = ['Expected shipping date']
df_3 = pd.DataFrame()
for m in time:
    for n in time:
        df_3.loc['ExpectedShippingDate '+ str(m), 'ActualShippingDate '+str(n)]=D[m,n].solution_value
print(df_3)
print("\n") 

                         ActualShippingDate 1  ActualShippingDate 2  \
ExpectedShippingDate 1                    0.0                 310.0   
ExpectedShippingDate 2                  332.0                  22.0   
ExpectedShippingDate 3                    0.0                   0.0   
ExpectedShippingDate 4                    0.0                   0.0   
ExpectedShippingDate 5                    0.0                   0.0   
ExpectedShippingDate 6                    0.0                   0.0   
ExpectedShippingDate 7                    0.0                   0.0   
ExpectedShippingDate 8                    0.0                   0.0   
ExpectedShippingDate 9                    0.0                   0.0   
ExpectedShippingDate 10                   0.0                   0.0   

                         ActualShippingDate 3  ActualShippingDate 4  \
ExpectedShippingDate 1                    0.0                   0.0   
ExpectedShippingDate 2                   26.0                   0.0   
Expec

Task 2 : Sensitivity Analysis

1. Sensitivity test with respect to SL

In [12]:
#Change the service level, SL
# Initialize a DataFrame to store the results of the sensitivity analysis
header_3 = ['Service Level','Total cost','Total normal Capacity cost','Total over Capacity cost']
output_3 = pd.DataFrame(columns = header_3)


k= -1
for SL in np.arange(0.8,1,0.01): # Loop over different values for SL to analyze the sensitivity with respect to SL

    k += 1
    # Define linear expressions
    Total_cost = model.sum(c_h * I[t] + NC * c_NC + O[t] * c_OC + B[t] * c_b for t in time)
    
    #Changed constraints
    model.add_constraints(L[t] <= (1-SL)* d_t[t-1] for t in time)
    
    # Define the objective function 
    model.minimize(Total_cost)

    # Solve the model
    model.solve()
    
    # Store the value in the output DataFrame
    output_3.loc[k,'Service Level'] = SL
    output_3.loc[k,'Total cost'] = model.objective_value
    
    # Store the total Normal Capacity cost and total Over Capacity cost in the output DataFrame
    Tc_NC = 0
    Tc_OC = 0
    for t in time:
        Tc_NC += NC.solution_value * c_NC
        Tc_OC += O[t].solution_value * c_OC
        output_3.loc[k, 'Total normal Capacity cost'] = round(Tc_NC) 
        output_3.loc[k, 'Total over Capacity cost'] = round(Tc_OC)
        
print(output_3)


   Service Level Total cost Total normal Capacity cost  \
0            0.8      39046                      33200   
1           0.81      39046                      33200   
2           0.82      39046                      33200   
3           0.83      39046                      33200   
4           0.84      39046                      33200   
5           0.85      39046                      33200   
6           0.86      39046                      33200   
7           0.87      39046                      33200   
8           0.88      39046                      33200   
9           0.89      39046                      33200   
10           0.9      39046                      33200   
11          0.91      39046                      33200   
12          0.92      39046                      33200   
13          0.93      39046                      33200   
14          0.94      39046                      33200   
15          0.95      39046                      33200   
16          0.

2. Sensitivity test with respect to (GT-,GT+)

In [13]:
#Change the goal time window (GT-,GT+)
# Initialize a DataFrame to store the results of the sensitivity analysis
header_4 = ['(GT-,GT+)','Total cost','Total normal Capacity cost','Total over Capacity cost']
output_4 = pd.DataFrame(columns = header_4)

SL = 0.95

k= -1
# Loop over different values for goal time window to analyze the sensitivity with respect to GTW
for GTL in range(1,6):
    for GTU in range(1,6): 
    
        k += 1
        
        # Define linear expressions
        Total_cost = model.sum(c_h * I[t] + NC * c_NC + O[t] * c_OC + B[t] * c_b for t in time)
        
        # Define the objective function 
        model.minimize(Total_cost)

        # Solve the model
        model.solve()
        
        
        # Store the value in the output DataFrame
        output_4.loc[k,'(GT-,GT+)'] = '('+str(GTL)+','+str(GTU)+')'
        output_4.loc[k,'Total cost'] = model.objective_value

        # Store the total Normal Capacity cost and total Over Capacity cost in the output DataFrame
        for t in time:
            Tc_NC = 0
            Tc_OC = 0
            Tc_NC += NC.solution_value * c_NC
            Tc_OC += O[t].solution_value * c_OC
            output_4.loc[k, 'Total normal Capacity cost'] = round(Tc_NC) 
            output_4.loc[k, 'Total over Capacity cost'] = round(Tc_OC)
        
print(output_4)


   (GT-,GT+) Total cost Total normal Capacity cost Total over Capacity cost
0      (1,1)      39580                       3370                        0
1      (1,2)      39580                       3370                        0
2      (1,3)      39580                       3370                        0
3      (1,4)      39580                       3370                        0
4      (1,5)      39580                       3370                        0
5      (2,1)      39580                       3370                        0
6      (2,2)      39580                       3370                        0
7      (2,3)      39580                       3370                        0
8      (2,4)      39580                       3370                        0
9      (2,5)      39580                       3370                        0
10     (3,1)      39580                       3370                        0
11     (3,2)      39580                       3370                        0
12     (3,3)